<a href="https://colab.research.google.com/github/indiradatta5/Profit-Prophet/blob/main/profit_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit
!pip install panel hvplot
!pip install streamlit-option-menu yfinance pandas matplotlib pyngrok
!pip install ta
!pip install prophet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=a59723b2d922d04a0bea90725e74c74418f87daf6c23a41ff6dd2a9e7bc441dc
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
from datetime import date
from plotly import graph_objs as go
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.diagnostics import cross_validation, performance_metrics
import time
import os
from streamlit_option_menu import option_menu

st.set_page_config(layout="wide", initial_sidebar_state="expanded")

# Call add_meta_tag immediately after setting the page configuration

# Meta tag function
def add_meta_tag():
    meta_tag = """
        <head>
            <meta name="google-site-verification" content="QBiAoAo1GAkCBe1QoWq-dQ1RjtPHeFPyzkqJqsrqW-s" />
        </head>
    """
    st.markdown(meta_tag, unsafe_allow_html=True)
add_meta_tag()

# Sidebar configuration
today = date.today()
st.write('''# Portfolio Performance Analysis ''')
st.sidebar.image("https://www.portseido.com/blog_images/portfolio-performance-evaluation/portfolio-performance-evaluation-cover.png", width=250, use_column_width=False)
st.sidebar.write('''# Portfolio Performance Analysis ''')

with st.sidebar:
    selected = option_menu("Menu", ["Stocks Performance Comparison", "Real-Time Stock Price", "Stock Price Prediction", 'About'],
                           icons=["house", "graph-up", "graph-down", "line-chart", "info-circle"],
                           menu_icon="cast", default_index=0)

start = st.sidebar.date_input('Start', datetime.date(2022, 1, 1))
end = st.sidebar.date_input('End', datetime.date.today())

# Load or initialize session state
if 'portfolio_data' not in st.session_state:
    if os.path.exists("portfolio_data.csv"):
        st.session_state['portfolio_data'] = pd.read_csv("portfolio_data.csv")
    else:
        st.session_state['portfolio_data'] = pd.DataFrame(columns=["Ticker", "Weight", "Buy Price"])

# Initialize session state
if 'page' not in st.session_state:
    st.session_state['page'] = 'comparison'
if 'portfolio_data' not in st.session_state:
    st.session_state['portfolio_data'] = pd.DataFrame(columns=['Ticker', 'Weight', 'Buy Price'])

# Function to save portfolio data
def save_portfolio_data(tickers, weights, buy_prices):
    st.session_state['portfolio_data'] = pd.DataFrame({
        'Ticker': tickers,
        'Weight': weights,
        'Buy Price': buy_prices
    })

# Function to get user inputs
def get_user_inputs():
    st.subheader("Enter your portfolio details")
    tickers = st.text_input("Enter tickers (comma separated)", "AAPL, MSFT, GOOGL").split(", ")
    weights = list(map(float, st.text_input("Enter weights (comma separated, should sum to 1.0)", "0.33, 0.33, 0.34").split(", ")))
    buy_prices = list(map(float, st.text_input("Enter buy prices (comma separated)", "150, 250, 2800").split(", ")))

    if st.button("Save Portfolio Data"):
        save_portfolio_data(tickers, weights, buy_prices)
        st.success("Portfolio data saved!")
        st.session_state['page'] = 'comparison'
        st.rerun()

# Function to edit user inputs
def edit_user_inputs():
    st.subheader("Edit your portfolio details")
    tickers = st.text_input("Edit tickers (comma separated)", ",".join(st.session_state['portfolio_data']['Ticker'].tolist())).split(", ")
    weights = list(map(float, st.text_input("Edit weights (comma separated, should sum to 1.0)", ",".join(map(str, st.session_state['portfolio_data']['Weight'].tolist()))).split(", ")))
    buy_prices = list(map(float, st.text_input("Edit buy prices (comma separated)", ",".join(map(str, st.session_state['portfolio_data']['Buy Price'].tolist()))).split(", ")))

    if st.button("Update Portfolio Data"):
        save_portfolio_data(tickers, weights, buy_prices)
        st.success("Portfolio data updated!")
        st.session_state['page'] = 'comparison'
        time.sleep(3)
        st.rerun()

# Display user inputs if data exists, otherwise prompt for inputs
if selected == 'Stocks Performance Comparison':
    if st.session_state['portfolio_data'].empty:
        get_user_inputs()
    else:
        if st.session_state['page'] == 'comparison':
            edit_mode = st.checkbox("Edit Portfolio Data")

            if edit_mode:
                st.session_state['page'] = 'edit'
                st.rerun()
            else:
                st.subheader("Your Portfolio")
                st.write(st.session_state['portfolio_data'])

                tickers = st.session_state['portfolio_data']['Ticker'].tolist()
                weights = st.session_state['portfolio_data']['Weight'].tolist()
                buy_prices = st.session_state['portfolio_data']['Buy Price'].tolist()

                def relativeret(df):
                    rel = df
                    rel = df.pct_change()
                    cumret = (1 + rel).cumprod() - 1
                    cumret = cumret.fillna(0)
                    return cumret

                def weighted_ret(df, weights):
                    weighted = (df * weights).sum(axis=1)
                    weighted = weighted.pct_change().fillna(0)
                    weighted_cumret = (1 + weighted).cumprod() - 1
                    return weighted_cumret

                if len(tickers) > 0:
                    with st.spinner('Loading...'):
                        df = relativeret(yf.download(tickers, start=start, end=end)['Adj Close'])
                        raw_df = relativeret(yf.download(tickers, start=start, end=end))
                        raw_df.reset_index(inplace=True)
                        closing_price = yf.download(tickers, start=start, end=end)['Adj Close']
                        volume = yf.download(tickers, start=start, end=end)['Volume']

                        st.subheader('Raw Data {}'.format(tickers))
                        st.write(raw_df)
                        chart = ('Line Chart', 'Area Chart', 'Bar Chart')
                        dropdown1 = st.selectbox('Pick your chart', chart)
                        with st.spinner('Loading...'):
                            time.sleep(2)

                        st.subheader('Relative Returns {}'.format(tickers))

                        if dropdown1 == 'Line Chart':
                            st.line_chart(df)
                            st.write("### Closing Price of {}".format(tickers))
                            st.line_chart(closing_price)
                            st.write("### Volume of {}".format(tickers))
                            st.line_chart(volume)
                        elif dropdown1 == 'Area Chart':
                            st.area_chart(df)
                            st.write("### Closing Price of {}".format(tickers))
                            st.area_chart(closing_price)
                            st.write("### Volume of {}".format(tickers))
                            st.area_chart(volume)
                        elif dropdown1 == 'Bar Chart':
                            st.bar_chart(df)
                            st.write("### Closing Price of {}".format(tickers))
                            st.bar_chart(closing_price)
                            st.write("### Volume of {}".format(tickers))
                            st.bar_chart(volume)
                        else:
                            st.line_chart(df)
                            st.write("### Closing Price of {}".format(tickers))
                            st.line_chart(closing_price)
                            st.write("### Volume of {}".format(tickers))
                            st.line_chart(volume)

                        # Weighted Returns Comparison
                        st.subheader("Weighted Returns Over Time")
                        selected_tickers = st.multiselect("Select tickers for comparison", options=tickers, default=tickers)
                        weights_input = st.text_input("Enter weights (comma separated, should sum to 1.0)", ", ".join(map(str, weights)))

                        if selected_tickers and weights_input:
                            selected_weights = list(map(float, weights_input.split(", ")))
                            if len(selected_weights) != len(selected_tickers):
                                st.error("Number of weights must match number of tickers")
                            elif sum(selected_weights) != 1.0:
                                st.error("Weights must sum to 1.0")
                            else:
                                with st.spinner('Loading...'):
                                    selected_data = yf.download(selected_tickers, start=start, end=end)['Adj Close']
                                    weighted_returns = weighted_ret(selected_data, selected_weights)

                                    st.subheader("Weighted Returns Over Time")
                                    st.line_chart(weighted_returns)
                else:
                    st.write('Please select at least one asset')
        elif st.session_state['page'] == 'edit':
            edit_user_inputs()

# Real-Time Stock Price Section
elif selected == 'Real-Time Stock Price':
    st.subheader("Real-Time Stock Price")
    ticker = st.text_input("Enter ticker", "AAPL")
    if ticker:
        with st.spinner('Loading...'):
            data = yf.download(ticker, start=start, end=end)
            data.reset_index(inplace=True)
            st.subheader(f'Raw Data of {ticker}')
            st.write(data)

            def plot_raw_data():
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
                fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
                fig.layout.update(title_text=f'Line Chart of {ticker}', xaxis_rangeslider_visible=True)
                st.plotly_chart(fig)

            def plot_candle_data():
                fig = make_subplots(rows=1, cols=1)
                fig.add_trace(go.Candlestick(x=data['Date'],
                                             open=data['Open'],
                                             high=data['High'],
                                             low=data['Low'],
                                             close=data['Close'], name=f'{ticker} stock'))
                fig.layout.update(title_text=f'Candlestick Chart of {ticker}', xaxis_rangeslider_visible=True)
                st.plotly_chart(fig)

            chart_types = ('Candlestick Chart', 'Line Chart')
            dropdown = st.selectbox('Pick your chart', chart_types)
            with st.spinner('Loading...'):
                time.sleep(2)
            if dropdown == 'Line Chart':
                plot_raw_data()
            elif dropdown == 'Candlestick Chart':
                plot_candle_data()

# Stock Price Prediction Section using Prophet
elif selected == "Stock Price Prediction":
    st.subheader("Stock Price Prediction")
    ticker = st.text_input("Enter ticker for prediction", "AAPL")
    periods_input = st.number_input("Enter the number of days for prediction", min_value=1, max_value=365)

    if ticker:
        with st.spinner('Loading historical data...'):
            data = yf.download(ticker, start=start, end=end)
            data.reset_index(inplace=True)

            # Prepare data for Prophet
            df_train = data[['Date', 'Close']]
            df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

            # Fit Prophet Model
            m = Prophet()
            m.fit(df_train)

            # Make future predictions
            future = m.make_future_dataframe(periods=periods_input)
            forecast = m.predict(future)

            st.subheader(f'Forecast data for {ticker}')
            st.write(forecast.tail())

            # Plot forecast data
            fig1 = plot_plotly(m, forecast)
            st.plotly_chart(fig1)

            # Plot forecast components
            st.write("Forecast components")
            fig2 = m.plot_components(forecast)
            st.write(fig2)

            # Model performance metrics
            st.write("Model Performance Metrics")
            df_cv = cross_validation(m, initial='365 days', period='90 days', horizon='180 days')
            df_p = performance_metrics(df_cv)
            st.write(df_p)

# About Section
elif selected == 'About':
    st.title("About This App")
    st.write("""
    The Portfolio Performance Analysis app allows users to monitor and predict the performance of their stock portfolio.
    It offers real-time stock prices, stock comparison analysis, and advanced machine learning predictions.
    Created by Rathan Rishi Mohanasundaram and Indira Datta (June 2024)
    """)



Overwriting app.py


In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.221.26.23


In [ ]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

your url is: https://fluffy-peas-punch.loca.lt
^C
